In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import quantecon as qe
from numba import jit
from scipy.optimize import brentq

In [2]:
#preference parameters

mu = 0.34
#sigma=2 would make this negative, so I'm betting it's a typo
sigma = 0.01
beta = 0.96

#technology parameters

theta = 0.36
delta = 0.025
rho = 0.9
#a standard deviation of 0.01 makes sense, but 2 would make the problem negative which we clearly don't want
sigma_sd = 2

#number of z

nz = 5

In [3]:
#get the probabilities from the Markov process

z_list = []
for i in list(range(1,nz+1)):
    z_list.append(f"z_{i}")

#this is the transition matrix. the rows are current states, and the columns are next period's states.
#interpretation: the ij entry is the probability of being in state i in the current period and going state to j next period 
#this makes sense because the ij entry where i=j is always the highest (probability of staying in the same state)
trans_matrix = pd.DataFrame(qe.markov.tauchen(rho, sigma, 0, sigma_sd, nz).P,
                     columns = z_list,
                     index = z_list)
print(trans_matrix)

#this gives us the states values of each z_i
state_values = pd.DataFrame(qe.markov.tauchen(rho, sigma, 0, sigma_sd, nz).state_values,
                            index = z_list,
                            columns = ['z_state_values'])
print(state_values)

              z_1           z_2       z_3           z_4           z_5
z_1  7.543514e-01  2.442186e-01  0.001430  6.581502e-08  1.854072e-14
z_2  8.433431e-02  7.362680e-01  0.178738  6.594660e-04  1.835626e-08
z_3  2.895316e-04  1.253850e-01  0.748651  1.253850e-01  2.895316e-04
z_4  1.835626e-08  6.594660e-04  0.178738  7.362680e-01  8.433431e-02
z_5  1.855818e-14  6.581502e-08  0.001430  2.442186e-01  7.543514e-01
     z_state_values
z_1       -0.045883
z_2       -0.022942
z_3        0.000000
z_4        0.022942
z_5        0.045883


In [64]:
#solve leisure
def l(kz):
    return brentq(lambda x: (x**(1-theta))*((1-mu)/mu+1-theta-(1-theta)/x)-kz, 0.00001, .9999999)

#brentq finds a zero of a nonlinear function. the formula in l(kz) uses kz = LHS from slide 7 and subtracts it from the RHS
#that is, kz = ((k'-(1-delta)k)/(zk^theta))

Basically we're idiots. We can't solve the steady state deterministically like we tried because z is stochastic. Slide 3 was if z = 1 always, but our z is random. Instead, we need to use Markov to find possible values and combine that with k (slide 7 formula) to find utility u(c,l). That is, we don't need to alter the code to include l; rather, we should alter it only to solve k with the possible values of z and the given parameters of our proble. 

Cleaned the leisure stuff out of the ramsey function and put it into the returns function

In [48]:
#this is the hw2_ex_v3 code from Rajesh--edit to fit our needs
"""parameters: mu, sigma, beta, theta, delta, rho, sigma_sd"""

class Ramsey_Econ602:

    def __init__(self,
                 mu,
                 sigma,
                 beta,
                 theta,
                 delta,
                 rho,
                 sigma_sd,
                 z,
                 nk=100,
                 nl=100,
                 tol=1e-10,
                 maxit=10000):

        # Save parameters
        """self.β, self.σ, self.α, self.δ, self.γ = β, σ, α, δ, γ
        self.nk = nk"""
        
        self.mu = mu
        self.sigma = sigma
        self.beta = beta
        self.theta = theta
        self.delta = delta
        self.rho = rho
        self.sigma_sd = sigma_sd,
        self.z = z
        self.nk = nk
        
        
        # Create grids and discretize state space
        
        
        """self.ks = (α*γ)**(1/(1-α))*(1/β-1+δ)**(1/(α-1)) 
        #ks = self.ks"""
        
        #steady state k and l (capital and leisure)
        self.ls = (1-theta)/(((1-mu)/mu)+1-theta-((delta*theta)/(1/beta-(1-delta))))
        self.ks = self.ls*((1/beta-(1-delta))/theta)**(1/theta-1)
        
        ks = self.ks
        ls = self.ls
    
        """self.kgrid = np.linspace(0.5*self.ks, 2*self.ks, nk)
        #self.kgrid = np.linspace(1, 8, nk)"""
        
        #need self.kgrid and lgrid
        self.kgrid = np.linspace(0.5*self.ks, 2*self.ks, nk)

        # Allocate memory for return, value, and policy functions
    
        self.v = np.zeros((nk))
        self.gk_index = np.empty((nk))
        self.gpr = np.empty((nk))
        self.Ret = np.zeros((nk,nk))
      
        # compute the return function
        _compute_returns(self.kgrid,self.delta,self.z,self.theta,self.mu,self.sigma,self.Ret)
                
        # Compute the value function and policy function
        
        self.solve(tol=tol, maxit=maxit)
        self.kpr_policy()
   
    # Below is the value function iteration function
        
    def solve(self, tol=1e-9, maxit=10000):
        # Iteration Stuff
        it = 0
        dist = 10.

        # Alloc memory to store next iterate of value function
        v_upd = np.zeros((self.nk))
        
        
       
        # Main loop
        while dist > tol and maxit > it:
            
            # Do the inner loop first
            _inner_loop(self.Ret,self.v,self.beta)
            
            dist = np.max(np.abs(v_upd - self.v))
            v_upd[:] = self.v[:]

            it += 1
            if it % 25 == 0:
                print(f"Running iteration {it} with dist of {dist}")

        return None    
        
    # This is to recover policy function after the value function has converged
    
    def kpr_policy(self):
    
    # Compute gpr and its index: best index in kgrid given ik  
        self.gpr = np.empty((self.nk))

        _compute_savings_policy(self.kgrid, self.Ret, self.v,
                                self.beta, self.gpr)

In [49]:
# Just to compute utility--need to solve for l before this can work        
@jit(nopython=True)
def u(c, l, mu, sigma):
    return (1/(1-sigma))((c^mu)((1-l)^(1-mu)))^(1-sigma)

In [50]:
@jit(nopython=True)
def _inner_loop(Ret,v,beta):
    
    """
    This is a numba version of the inner loop of the solve in the
    Ramsey class. It updates v in place.
    """
    nk = len(v)
    for ik in range(nk):
        current_max = -1e14
        for ik_next in range(nk):
            m = Ret[ik,ik_next] + beta * v[ik_next]
            
            if m > current_max:
                current_max = m      
                
            v[ik] = current_max  
            
    return None

In [51]:
@jit(nopython=True)
def _compute_savings_policy(kgrid, Ret,v,beta, gpr):
    # Compute best k'
    nk = len(kgrid)
    for ik in range(nk):
        current_max = -1e14

        for ik_next in range(nk):
            m = Ret[ik,ik_next] + beta * v[ik_next]
            if m > current_max:
                current_max = m
                current_max_index = ik_next
            gpr[ik] = kgrid[current_max_index]
    return None 

In [84]:
#@jit(nopython=True)
def _compute_returns(kgrid,delta,z,theta,mu,sigma,Ret):
    # Compute the return given k and k'
    nk = len(kgrid)
    for i in range(nk):
        for j in range(nk):
            #add leisure solution here
            kz = ((kgrid[j]-(1-delta)*kgrid[i])/(z*kgrid[i]**theta))
            print(f"kz (LHS): {round(kz,6)}")
            leisure = l(kz)
            print(f"leisure: {round(leisure,6)}")
            #our budget constraint is c = zf(k,l)-k'+(1-d)k and f(k,l) = (k^theta)(l^(1-theta))
            c = z*(kgrid[i]**theta)*(leisure**(1-theta))-kgrid[j]+(1-delta)*kgrid[i]
            if c <= 0:
                Ret[i,j] = -1e+14
            else: 
                Ret[i,j] = u(c, leisure, mu, sigma)
    return None

In [85]:
for state in state_values.z_state_values:
    print(f"state: {round(state,6)}")
    z = state
    Ramsey_Econ602(mu,sigma,beta,theta,delta,rho,sigma_sd,z)

state: -0.045883
kz (LHS): -1.526257
leisure: 0.048714
kz (LHS): -3.376265
leisure: 0.008905
kz (LHS): -5.226273
leisure: 0.002836
kz (LHS): -7.076282
leisure: 0.001244
kz (LHS): -8.92629
leisure: 0.000657
kz (LHS): -10.776298
leisure: 0.000391
kz (LHS): -12.626306
leisure: 0.000252
kz (LHS): -14.476315
leisure: 0.000172
kz (LHS): -16.326323
leisure: 0.000124
kz (LHS): -18.176331
leisure: 9.2e-05
kz (LHS): -20.026339
leisure: 7e-05
kz (LHS): -21.876348
leisure: 5.5e-05
kz (LHS): -23.726356
leisure: 4.4e-05
kz (LHS): -25.576364
leisure: 3.6e-05
kz (LHS): -27.426372
leisure: 2.9e-05
kz (LHS): -29.276381
leisure: 2.4e-05
kz (LHS): -31.126389
leisure: 2.1e-05
kz (LHS): -32.976397
leisure: 1.8e-05
kz (LHS): -34.826406
leisure: 1.5e-05
kz (LHS): -36.676414
leisure: 1.3e-05
kz (LHS): -38.526422
leisure: 1.1e-05
kz (LHS): -40.37643
leisure: 1e-05
kz (LHS): -42.226439


ValueError: f(a) and f(b) must have different signs